In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import confusion_matrix, make_scorer
import xgboost as xgb

In [4]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 2)

In [5]:
data = pd.read_csv('./Data/training_data.csv', sep=";", decimal=',')
data.head()

,Group,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11,I12,I13,I14,I15,I16,I17,I18,I19,I20,I21,I22,I23,I24,I25,I26,I27,I28,I29,I30,I31,I32,I33,I34,I35,I36,I37,I38,I39,I40,I41,I42,I43,I44,I45,I46,I47,I48,I49,I50,I51,I52,I53,I54,I55,I56,I57,I58,dI1,dI2,dI3,dI4,dI5,dI6,dI7,dI8,dI9,dI10,dI11,dI12,dI13,dI14,dI15,dI16,dI17,dI18,dI19,dI20,dI21,dI22,dI23,dI24,dI25,dI26,dI27,dI28,dI29,dI30,dI31,dI32,dI33,dI34,dI35,dI36,dI37,dI38,dI39,dI40,dI41,dI42,dI43,dI44,dI45,dI46,dI47,dI48,dI49,dI50,dI51,dI52,dI53,dI54,dI55,dI56,dI57,dI58,Class,Perform
0,G9,0.136495,-0.028429,-0.037772,-0.232459,-0.016222,-0.187506,-0.322545,-0.043743,0.125389,-0.014757,-0.033105,0.303035,-0.093811,-0.598917,-0.271292,-0.256749,-0.100146,-0.045525,-0.078422,-0.060129,-0.069528,-0.052432,-0.114432,-0.104989,0.342845,-0.159417,0.006772,-0.303193,-0.163287,-0.080599,-0.828880,-1.064215,-0.547067,-0.540497,-0.676045,-0.305007,-0.507724,-0.191437,-0.087362,-0.856151,0.802525,0.733080,0.006512,0.533290,0.195197,0.058094,-0.228889,-0.150821,-0.104986,-0.026743,0.188312,-0.250701,-0.101190,-0.357521,-0.527956,0.611385,-0.092714,-0.055733,-0.065709,-0.002144,-0.004367,-0.079805,0.178280,0.078155,0.072802,0.002090,0.211770,-0.003073,-0.188447,0.117769,0.001613,-0.024223,0.103204,0.032484,0.002688,0.000765,-0.004447,0.148967,-0.018521,-0.014110,-0.001996,-0.002369,-0.120036,0.013172,-0.215571,-0.021999,0.001728,-0.000050,-0.012120,-0.040172,-0.060103,-0.059464,-0.044899,0.015735,0.022919,-0.003106,0.001233,-0.002339,0.040628,0.411684,0.073090,0.526222,0.071060,-0.019531,0.359889,-0.020476,0.057151,0.077110,0.102563,0.188481,-0.016027,-0.135451,-0.189667,0.250967,0.022171,-0.004265,-1,-0.033764
1,G5,-0.714522,-0.042137,-0.052968,-0.796862,-0.018394,0.070102,-0.076321,-0.063864,-1.045521,-0.037353,-0.792515,-1.082483,0.025798,-0.833652,-0.625088,-0.333608,0.072579,-0.046963,0.223022,-0.605902,-0.131099,-0.235929,-0.073920,-0.063247,-0.798768,-0.899983,1.388771,-0.248677,-0.058083,-0.014470,0.092095,0.561368,0.224819,0.223190,0.098852,-0.128227,-0.215876,-0.007164,-0.035260,-0.123911,-0.089751,-0.094963,0.362818,0.011107,-1.506356,-0.573679,-0.955222,-0.818880,-1.063295,-1.022679,-1.336188,-0.612039,-0.061357,-0.482805,-0.017077,1.192135,-0.114981,-0.028074,-0.004451,-0.000536,-0.002288,-0.045597,-0.080639,-0.081924,-0.033862,-0.005111,-0.261836,0.000122,-0.045046,0.999854,-0.008835,-0.122379,-0.199892,0.013615,0.014404,-0.000405,0.021573,-0.024160,-0.037420,-0.012610,0.003007,0.003617,-0.106893,-0.394834,-0.132496,-0.027354,-0.129804,-0.066157,-0.494334,0.123781,0.284328,0.281308,0.212767,0.192042,0.146926,-0.118826,-0.039203,-0.256107,0.176622,0.168840,0.487752,0.029464,0.014232,0.039633,0.025667,0.006626,0.005180,0.006128,-0.016375,0.020727,-0.006525,-0.018790,-0.098543,0.317744,-0.180502,-0.009215,1,0.127771
2,G10,0.104791,-0.038188,-0.053191,0.620233,0.148587,0.489875,0.319274,-0.060246,0.053174,-0.025008,-0.456840,1.284450,-0.133470,3.207672,2.373230,1.304427,NaN,NaN,-0.361293,2.995661,NaN,-0.188988,-0.044158,-0.024550,-0.586562,-0.176292,-1.013037,0.066912,0.219649,0.154490,2.370951,1.384675,0.489152,0.484715,0.367301,0.749572,0.669410,0.423228,0.226897,3.227283,-0.329997,-0.327579,-1.033898,0.014531,0.211889,-1.197156,2.860444,NaN,3.584223,NaN,1.272375,7.427558,-0.182816,-2.713205,-1.877595,-0.568691,0.224945,0.052749,0.377640,0.002656,0.001226,0.226060,0.207653,0.270327,0.283061,0.002934,0.454366,0.004264,0.188623,-0.265918,0.000000,2.063796,1.076458,0.240011,NaN,NaN,-0.028327,1.764826,NaN,0.005847,-0.011166,-0.012626,-0.010822,0.056514,-0.100007,-0.216081,-0.127274,-0.056206,0.175751,-0.011770,0.493157,0.487919,0.438576,0.574623,0.564379,-0.165933,-0.051256,0.410379,0.056624,0.047592,0.000000,-0.020586,0.237539,0.017314,0.516667,NaN,0.404158,NaN,0.272937,0.774169,-0.007144,0.123954,0.000000,-0.110103,0.186669,-0.030720,1,0.183500
3,G2,-0.532847,-0.006582,-0.023377,1.306702,-0.068909,0.048024,-0.119481,-0.021057,-1.012916,-0.011783,1.206727,0.311773,-0.005928,3.869459,-1.064793,0.1

In [6]:
#imputing missing value and creating a dummy column associated with that
cols_with_missing= [col for col in data.columns if data[col].isnull().any()]

#dummy column indicating missing values
datacopy= data.copy()
for col in cols_with_missing:
    datacopy[col + '_missing']= datacopy[col].isnull().astype(int)

features=datacopy.drop(columns = ['Perform','Class'], axis=1)
target=datacopy['Class']

In [7]:
#train and test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=42)

X_train.index

Index([1467, 5768, 5714, 1578, 6958, 6861, 2126, 6202, 1450, 5067,
       ...
       6265, 5734, 3092, 3772, 5191, 5226, 5390,  860, 7603, 7270],
      dtype='int64', length=6400)

In [8]:
#imputing numerical columns
numerics=X_train.drop('Group', axis=1)
imputer= SimpleImputer()

#imputation
numeric_imp = pd.DataFrame(imputer.fit_transform(numerics))

#imputation removes columns names, put it back
numeric_imp.columns=numerics.columns

#applying to test set
numerics_test=X_test.drop('Group', axis=1)

#imputation
numeric_test_imp = pd.DataFrame(imputer.transform(numerics_test))

#imputation removes columns names, put it back
numeric_test_imp.columns=numerics_test.columns

In [9]:
#encoding the categorical variable Group
categorical=X_train['Group']
encoder= OneHotEncoder(handle_unknown='ignore', drop='first')
categorical_imp=encoder.fit_transform(categorical.values.reshape(-1,1)).toarray()
one_hot_df = pd.DataFrame(categorical_imp, columns = encoder.get_feature_names_out(['Industry']))

#applying to test data
categorical_test=X_test['Group']
categorical_test_imp=encoder.transform(categorical_test.values.reshape(-1,1)).toarray()
one_hot_test=pd.DataFrame(categorical_test_imp, columns = encoder.get_feature_names_out(['Industry']))

In [23]:
#scaling numerical variables
scaler=StandardScaler()

#we dont want to scale dummy variables
numerical_cols = [col for col in numeric_imp.columns if numeric_imp[col].nunique() > 2]
print(len(numerical_cols))
numerical_cols_test = [col for col in numeric_test_imp.columns if numeric_test_imp[col].nunique() > 2]

#scaling train data
scaled_numeric= pd.DataFrame(scaler.fit_transform(numeric_imp[numerical_cols]), columns=numerical_cols, index=numeric_imp.index)

#scaling test data
scaled_numeric_test= pd.DataFrame(scaler.transform(numeric_test_imp[numerical_cols_test]), columns=numerical_cols_test, index=numeric_test_imp.index)

116


In [213]:
#combining scaled data set with dummy variables for missing values and one hot encoding for Group
#train_features= pd.concat([scaled_numeric, numeric_imp.drop(columns=numerical_cols), one_hot_df], axis=1)
#test_features= pd.concat([scaled_numeric_test, numeric_test_imp.drop(columns=numerical_cols_test), one_hot_test], axis=1)

In [11]:
#joinning train data as well as test data
train_features=pd.concat([numeric_imp, one_hot_df],axis=1)
test_features=pd.concat([numeric_test_imp, one_hot_test],axis=1)

In [12]:
# Define the expected class mapping
label_mapping = {0: 2, 1: 0, -1 : 1}  # Change according to your expected class order

# Apply the mapping to the target variable
y_transformed = y_train.map(label_mapping)

# Ensure the transformed classes match the expected ones
print("Unique transformed classes:", y_transformed.unique())  # Should be [0, 1, 2]


Unique transformed classes: [0 1 2]


In [13]:
# Get unique classes and their respective weights
class_weights = compute_class_weight('balanced', classes=y_transformed.unique(), y=y_transformed)

# Create a dictionary with classes as keys and their respective weights as values
class_weight_dict = dict(zip(y_transformed.unique(), class_weights))

print(class_weight_dict)

{0: 0.7096917276557995, 1: 0.8623012665049852, 2: 2.318840579710145}


In [14]:
def avg_err_cost(y_true, y_pred):
    # Define the desired rows
    row1 = [0, 1, 2]
    row2 = [1, 0, 1]
    row3 = [2, 1, 0]

    cost_matrix = np.array([row1, row2, row3]) 
    confu_matrix = confusion_matrix(y_true, y_pred)
    err = np.sum(confu_matrix * cost_matrix)/len(y_true)
    return  err

In [15]:
custom_scorer = make_scorer(avg_err_cost)

In [16]:
y_transformed.shape

(6400,)

In [21]:
# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# For multiclass, you'd typically use sample weights based on class_weight_dict
sample_weights = y_transformed.map(class_weight_dict)


# Initialize XGBoost classifier with class weights
xgb_classifier = xgb.XGBClassifier(
    objective='multi:softmax',  # Use 'multi:softmax' or 'multi:softprob' for multiclass
    num_class = 3  # Adjust for binary or multiclass
)

# Fit XGBoost with sample weights to address class imbalance
xgb_classifier.fit(train_features, y_transformed, sample_weight=sample_weights)

# Apply cross-validation
cv_results = cross_val_score(
    xgb_classifier, train_features, y_transformed, cv=skf, scoring=custom_scorer, n_jobs=-1
)

print('Cross-Validation Scores:', cv_results)
print('Mean Score:', cv_results.mean())

Cross-Validation Scores: [0.63515625 0.65703125 0.640625   0.64765625 0.60625   ]
Mean Score: 0.63734375
